## **갤러리아 보고 (2022.10.06)**

---

### **분석 목적** 
* 타겟 고객군의 등급 상향 이동 여부 분류 데이터 분석 
* 신규 고객 온보딩 분석 
* 향후 우수고객 전환 관련 데이터 분석 (1번과 중복되는 것 같음)
* 고객 이탈분석 및 이탈관련 index 화 (고객 프로파일링)
* look-alike 타겟팅 (갤러리아 외 고객의 데이터 받을 수 있는가?)


### **1. 갤러리아 데이터 불러오기**
* 1-1) data read 하기 
* 1-2) 데이터 결합하기

### **2. EDA**

* 2-1) feature engineering  

### **3. 데이터 핸들링**

* 3-1) Missing Imputation 
* 3-2) feature selection 

### **4. 모형 해석**
* 3-1) Paramteric model
* 3-2) Assumption check  

### **4. Machine Laerning**
* 4-1) Ensemble 
* 4-2) Neural Net 
* 4-3) Prediction Performance Result


### **5. Reference** 
* ref 


----


In [32]:
import os 
import sys 
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 
import re 
from pandas_profiling import ProfileReport
import openpyxl
import plotly.offline as plyo
import cufflinks
import warnings


---
* 분석 환경설정 


In [12]:
# Jupyter 환경 설정 

warnings.filterwarnings(action='ignore') 
plt.rcParams['font.family'] = 'AppleGothic'
file_path = '/Users/dk/Downloads/scheme'
os.chdir(file_path)


---

### **1. 갤러리아 데이터 불러오기**
* 1-1) data read 하기 


In [34]:
file_nm = [x for x in os.listdir() if 'xlsx' in x]
file_nm 

['04. GM_IND_MEM_개인회원정보_20220922.xlsx',
 '12. PS_CUST_DAILY_MEMO_PSR 일지(일별 메모)_20220922.xlsx',
 '16. PS_SALE_PSR매출_20220922.xlsx',
 '97.CULT_TR부분접수결제환불.xlsx',
 '03. GM_GRADE_TARGET_회원 등급 최종 대상자_20220921 (PJ추출).xlsx',
 '13. PS_CUST_RESRV_PSR고객예약접수_20220922.xlsx',
 '96.CULT_CUST_MAST문화회원마스터.xlsx',
 '98.CULT_BABY유모차대여.xlsx',
 '99.VIP_WINE상반기 와인 구독 신청 회원번호.xlsx',
 '05. GM_REPRE_MEM_INFO_대표회원정보_20220922.xlsx',
 '15. PS_INVITE_REG_PSR 출입등록 관리_20220922.xlsx',
 '14. PS_FANCY_PSR고객취향_20220922.xlsx',
 '95.PSR 메모 텍스트마이닝 DATA.xlsx']

* file name 정규화

In [26]:
def reg_file_nm(file_nm):
    
    return re.sub(
        '[_]+$',
        '',
        re.sub(
            '[ㄱ-ㅎ가-힣0-9\.\(\) ]',
            '',
            re.sub(
                '.xlsx',
                '',
                file_nm
            )   
        )
    ) 

* 파일 이름 핸들링 결과

In [35]:
for nm in file_nm: 
	print(reg_file_nm(nm))
	print('-'*100)

GM_IND_MEM_개인회원정보
----------------------------------------------------------------------------------------------------
PS_CUST_DAILY_MEMO_PSR일지일별메모
----------------------------------------------------------------------------------------------------
PS_SALE_PSR매출
----------------------------------------------------------------------------------------------------
CULT_TR부분접수결제환불
----------------------------------------------------------------------------------------------------
GM_GRADE_TARGET_회원등급최종대상자_PJ추출
----------------------------------------------------------------------------------------------------
PS_CUST_RESRV_PSR고객예약접수
----------------------------------------------------------------------------------------------------
CULT_CUST_MAST문화회원마스터
----------------------------------------------------------------------------------------------------
CULT_BABY유모차대여
--------------------------------------------------

* 데이터 불러오기

In [ ]:
galleria_db = {}

for nm in file_nm:

    xlsx_dt = pd.read_excel(
        nm,
        None
    )
    sheet_nm = list(xlsx_dt.keys())[0]

    xlsx_df = pd.DataFrame(
        xlsx_dt[sheet_nm]
    )

    galleria_db[reg_file_nm(nm)] = xlsx_df
    
    print(nm)
    print('-'*100)
    
tbl_list = list(galleria_db.keys())

---
* 컬럼에 한글명 붙이기

#### 개인회원 정보 테이블 (GM_IND_MEM) **base table**

In [47]:
galleria_db['GM_IND_MEM_개인회원정보'].head()

,MEM_NO,CREDIT_CARD_FIRST_JOIN_YMD,MEMSHIP_JOIN_YMD,FOREINER_FLAG_CD,FOREINER_FLAG_NM,EMAIL_RECV_AGREE_YN,SMS_AGREE_YN,PARKJADE_CHOICE_RSN_CD,PARKJADE_CHOICE_RSN_NM,DM_RECV_AGREE_YN,TM_AGREE_YN,DM_ADDR,DM_OFFICE_NM,DM_DEPT_NM,INET_EMAIL_RECV_AGREE_YN,INET_SMS_RECV_AGREE_YN
0,12367029,2017-12-02,NaN,1.0,내국인,N,Y,NaN,NaN,Y,N,NaN,NaN,NaN,N,Y
1,12376469,NaN,2017-12-31,4.0,외국인등록증,N,N,NaN,NaN,N,N,NaN,NaN,NaN,N,N
2,12377168,2019-11-09,2018-01-04,1.0,내국인,Y,Y,NaN,NaN,N,N,NaN,NaN,NaN,Y,Y
3,12378037,2018-01-07,NaN,1.0,내국인,Y,Y,NaN,NaN,Y,Y,NaN,NaN,NaN,Y,Y
4,12684462,2022-07-15,2020-01-16,1.0,내국인,N,N,NaN,NaN,N,N,NaN,NaN,NaN,N,N


* Missing 이 80프로 이상이거나, 카테고리가 1가지인 컬럼 Drop 하기

In [57]:
galleria_db['GM_IND_MEM_개인회원정보'].info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 168328 entries, 0 to 168327
Data columns (total 16 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   MEM_NO                      168328 non-null  int64  
 1   CREDIT_CARD_FIRST_JOIN_YMD  124889 non-null  object 
 2   MEMSHIP_JOIN_YMD            108218 non-null  object 
 3   FOREINER_FLAG_CD            159048 non-null  float64
 4   FOREINER_FLAG_NM            159048 non-null  object 
 5   EMAIL_RECV_AGREE_YN         168326 non-null  object 
 6   SMS_AGREE_YN                168326 non-null  object 
 7   PARKJADE_CHOICE_RSN_CD      1186 non-null    float64
 8   PARKJADE_CHOICE_RSN_NM      1186 non-null    object 
 9   DM_RECV_AGREE_YN            168295 non-null  object 
 10  TM_AGREE_YN                 168295 non-null  object 
 11  DM_ADDR                     5449 non-null    object 
 12  DM_OFFICE_NM                4 non-null       object 
 13  DM_DEPT_NM    

In [60]:
# Drop col 선택 

galleria_db['GM_IND_MEM_개인회원정보'].drop(
	columns = [
		'PARKJADE_CHOICE_RSN_CD',
		'PARKJADE_CHOICE_RSN_NM',
		'DM_ADDR',
		'DM_OFFICE_NM',
		'DM_DEPT_NM'
	],
	inplace = True	
)

# 데이터 확인 
galleria_db['GM_IND_MEM_개인회원정보'].head()




,MEM_NO,CREDIT_CARD_FIRST_JOIN_YMD,MEMSHIP_JOIN_YMD,FOREINER_FLAG_CD,FOREINER_FLAG_NM,EMAIL_RECV_AGREE_YN,SMS_AGREE_YN,DM_RECV_AGREE_YN,TM_AGREE_YN,INET_EMAIL_RECV_AGREE_YN,INET_SMS_RECV_AGREE_YN
0,12367029,2017-12-02,NaN,1.0,내국인,N,Y,Y,N,N,Y
1,12376469,NaN,2017-12-31,4.0,외국인등록증,N,N,N,N,N,N
2,12377168,2019-11-09,2018-01-04,1.0,내국인,Y,Y,N,N,Y,Y
3,12378037,2018-01-07,NaN,1.0,내국인,Y,Y,Y,Y,Y,Y
4,12684462,2022-07-15,2020-01-16,1.0,내국인,N,N,N,N,N,N


* 년도별 회원등급 최종 대상자 PJ 추출 (GM_GRADE_TARGET)

In [48]:
galleria_db['GM_GRADE_TARGET_회원등급최종대상자_PJ추출'].head()

,REPRE_MEM_NO,SALE_YY,CHOICE_RSN_CD,CHOICE_RSN_NM,TOT_NET_SALE_AMT,GRADE_LAR_CD,GRADE_LAR_NM,GRADE_SMALL_CD,GRADE_SMALL_NM,MGM_STORE_CD,MGM_STORE_NM,CONCIGE_VOCH_USE_CNT,PJ_APPLY_YN
0,10138617,2017,1.0,매출기준,24976322.0,2,ParkJade,3,BLUE,1500.0,수원점,0,Y
1,10138629,2017,1.0,매출기준,43348725.0,2,ParkJade,2,WHITE,9100.0,명품관,0,Y
2,10138933,2017,1.0,매출기준,22522817.0,2,ParkJade,3,BLUE,2900.0,센터시티_BLUE,0,Y
3,10139084,2017,1.0,매출기준,31392979.0,2,ParkJade,3,BLUE,9100.0,명품관,0,Y
4,10139248,2017,1.0,매출기준,23455226.0,2,ParkJade,3,BLUE,5410.0,타임월드_BLUE,0,Y


In [54]:
s1 = set(galleria_db['GM_IND_MEM_개인회원정보'].MEM_NO) 
s2 = set(galleria_db['GM_GRADE_TARGET_회원등급최종대상자_PJ추출'].REPRE_MEM_NO)

print('개인회원 정보와 회원등급대상자 회원 ID 교집합 개수 :' ,len(s1 & s2))
print('신용카드 가입일과 매출년도는 별개로 봐도 무방한가?')

개인회원 정보와 회원등급대상자 회원 ID 교집합 개수 : 28516


---
* 데이터 결합 키 확인

---
* 데이터 결합 

---
* 데이터 결합 결과 

In [40]:
tbl_list

['GM_IND_MEM_개인회원정보',
 'PS_CUST_DAILY_MEMO_PSR일지일별메모',
 'PS_SALE_PSR매출',
 'CULT_TR부분접수결제환불',
 'GM_GRADE_TARGET_회원등급최종대상자_PJ추출',
 'PS_CUST_RESRV_PSR고객예약접수',
 'CULT_CUST_MAST문화회원마스터',
 'CULT_BABY유모차대여',
 'VIP_WINE상반기와인구독신청회원번호',
 'GM_REPRE_MEM_INFO_대표회원정보',
 'PS_INVITE_REG_PSR출입등록관리',
 'PS_FANCY_PSR고객취향',
 'PSR메모텍스트마이닝DATA']